In [0]:
import numpy as np
import random

def sigmoid(z):
    """
    The sigmoid activation function.
    """
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    """
    Derivative of the sigmoid function.
    """
    return sigmoid(z) * (1 - sigmoid(z))

def cost_function(network, test_data, onehot=True):
    """
    Целевая функция с усреднением  (1/n * (y_hat - y) ^ 2).
    """
    c = 0
    for example, y in test_data:
        # если ожидаемые значения представлены не в формате onehot, то переформатируем их 
        if not onehot:
            y = np.eye(3, 1, k =- int(y))
        yhat = network.feedforward(example)
        c += np.sum((y - yhat) ** 2)
    return c / len(test_data)

In [0]:
class Network:
    def __init__(self, shape, activation_function, activation_function_derivative, output=True,):
        """
        Список ``shape`` содержит количество нейронов в соответствующих слоях
        нейронной сети. К примеру, если бы этот лист выглядел как [2, 3, 1],
        то мы бы получили трёхслойную нейросеть, с двумя нейронами в первом
        (входном), тремя нейронами во втором (промежуточном) и одним нейроном
        в третьем (выходном, внешнем) слое. Смещения и веса для нейронных сетей
        инициализируются случайными значениями, подчиняющимися стандартному нормальному
        распределению. Обратите внимание, что первый слой подразумевается слоем, 
        принимающим входные данные, поэтому мы не будем добавлять к нему смещение 
        (делать это не принято, поскольку смещения используются только при 
        вычислении выходных значений нейронов последующих слоёв)
        """

        self.layers = len(shape)
        self.shape = shape
        self.biases = [np.random.randn(y, 1) for y in shape[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(shape[:-1], shape[1:])]
        self.output = output
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative

    def feedforward(self, input_matrix):
        """
        ``input_matrix`` - матрица входов размерностью (n, m)
        Вычислить и вернуть выходную активацию нейронной сети
        при получении ``input_matrix`` на входе (бывшее forward_pass).
        Возвращает матрицу (n, 1)
        """
        for b, w in zip(self.biases, self.weights):
            # weigts - (массив матриц)
            input_matrix = self.activation_function(np.dot(w, input_matrix) + b)
        return input_matrix

    def update_mini_batch(self, mini_batch, alpha):
        """
        Обновить веса и смещения нейронной сети, сделав шаг градиентного
        спуска на основе алгоритма обратного распространения ошибки, примененного
        к одному mini batch.
        ``mini_batch`` - список кортежей вида ``(x, y)`` x - вход, y - желаемый выход,
        ``alpha`` - величина шага (learning rate).
        """
        # значения dJ/db для каждого слоя
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        # значения dJ/dw (ошибки) для каждого слоя
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # для каждого примера из батча применяем бек пропогейшн
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        eps = alpha / len(mini_batch)
        # обновляем параметры сети
        self.weights = [w - eps * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]

    def evaluate(self, test_data):
        """
        Вернуть количество тестовых примеров, для которых нейронная сеть
        возвращает правильный ответ. Обратите внимание: подразумевается,
        что выход нейронной сети - это индекс, указывающий, какой из нейронов
        последнего слоя имеет наибольшую активацию. (случай для формата one-hot)
        """
        # кортеж полученных и ожидаемых значений для каждого примера тестовой выборки
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        # возвращаем количество верно угаданных целевых переменных
        return sum(int(x == y) for (x, y) in test_results)

    def SGD(self, training_data, epochs, mini_batch_size, alpha, test_data=None):
        """
        Обучить нейронную сеть, используя алгоритм стохастического
        (mini-batch) градиентного спуска. 
        ``training_data`` - лист кортежей вида ``(x, y)``, где 
        x - вход обучающего примера, y - желаемый выход (в формате one-hot). 
        Роль остальных обязательных параметров должна быть понятна из их названия.
        Если предоставлен опциональный аргумент ``test_data``, 
        то после каждой эпохи обучения сеть будет протестирована на этих данных 
        и промежуточный результат обучения будет выведен в консоль. 
        ``test_data`` -- это список кортежей из входных данных 
        и номеров правильных классов примеров (т.е. argmax(y),
        если y -- набор ответов в той же форме, что и в тренировочных данных).
        Тестирование полезно для мониторинга процесса обучения,
        но может существенно замедлить работу программы.
        """

        if test_data is not None: 
            n_test = len(test_data)

        n = len(training_data)
        success_tests = 0
        for j in range(epochs):
            random.shuffle(training_data)
            # выбираем несолько батчей, покрывающих всю тренировочную выборку
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            # обновляем параметры сети (веса, биасы) бекпропогейшном для каждого из батчей
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, alpha)
            if test_data is not None and self.output:
                success_tests = self.evaluate(test_data)
                print(f'Эпоха {j}: {success_tests} / {n_test}')
            elif self.output:
                pass
                # print(f'Эпоха {j} завершена')
        # в случае наличия тестовой выборки возвращаем количество правильных активаций сети
        if test_data is not None:
            return success_tests / n_test
    
    def cost_derivative(self, output_activations, y):
        """
        Возвращает вектор частных производных (\partial C_x) / (\partial a) 
        целевой функции по активациям выходного слоя. подходит для квадратичной функции 
        """
        return (output_activations-y)

    def backprop(self, x, y):
        """
        Возвращает кортеж ``(nabla_b, nabla_w)`` -- градиент целевой функции по всем параметрам сети.
        ``nabla_b`` и ``nabla_w`` -- послойные списки массивов ndarray,
        такие же, как self.biases и self.weights соответственно.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # прямое распространение (forward pass)
        activations = [x]
        summatories = []
        for b, w in zip(self.biases, self.weights):
            summatories.append(np.dot(w, activations[-1]) + b)
            activation = self.activation_function(summatories[-1])
            activations.append(activation)

        # обратное распространение (backward pass)
        
        # ошибка для выходного слоя
        delta = self.cost_derivative(activations[-1], y) * self.activation_function_derivative(summatories[-1])
        # производная J по биасам выходного слоя
        nabla_b[-1] = delta
        # производная J по весам выходного слоя
        nabla_w[-1] = delta.dot(activations[-2].T)

        # Здесь l = 1 означает последний слой, l = 2 - предпоследний и так далее.  
        for l in range(2, self.layers):
            derivative = self.activation_function_derivative(summatories[-l])
            # суммарные ошибки следующих слоев
            total_deltas = self.weights[-l + 1].T.dot(delta)
            # ошибка на слое L-l
            delta = derivative * total_deltas
            # производная J по смещениям L-l-го слоя
            nabla_b[-l] = delta
            # производная J по весам L-l-го слоя
            nabla_w[-l] = delta.dot(activations[-l - 1].T) 
        return nabla_b, nabla_w

    def test(self, test_data):
        for (x, y) in test_data:
            result = np.argmax(self.feedforward(x))
            print(f'result for {list(x)} is {result} should be: {y}')

In [0]:
training_data = [(np.array([[0], [0]]), 0), 
                 (np.array([[0], [1]]), 1), 
                 (np.array([[1], [0]]), 1), 
                 (np.array([[1], [1]]), 0)]
# XOR
network = Network([2, 2, 2], sigmoid, sigmoid_prime)
network.SGD(training_data, 100000, 4, 0.01)
network.test(training_data)

result for [array([0]), array([1])] is 0 should be: 1
result for [array([1]), array([1])] is 1 should be: 0
result for [array([1]), array([0])] is 1 should be: 1
result for [array([0]), array([0])] is 1 should be: 0


In [0]:
network = Network([3,2,1],sigmoid, sigmoid_prime)
network.biases = [np.array([[0], [0]]), np.array([[0]])]
network.weights = [np.array([[0.2, 0.9, 0.6], [0.2, 0.3,  0.7]]), np.array([[0.2, 0.5]])]

x = np.array([[15], [5], [15]])
y = np.array([[1]])
nabla_b, nabla_w = network.backprop(x, y)
nabla_w

shape activ (3, 1)
shape w (2, 3)
shape activ (2, 1)
shape w (1, 2)


[array([[-1.50641840e-08, -5.02139468e-09, -1.50641840e-08],
        [-1.68782392e-07, -5.62607975e-08, -1.68782392e-07]]),
 array([[-0.07356705, -0.07356703]])]

In [0]:
(sigmoid(0.9 * 0.2) - 1) * 0.2

-0.09102422152528399

In [0]:
network = Network([3,2], sigmoid, sigmoid_prime)
network.biases = [np.array([[-1], [-1]])]
network.weights = [np.array([[-1,1,-1], [1,-1,1]])]

x = np.array([[1], [2], [3]])
y = np.array([[0], [1]])

In [0]:
nabla_b, nabla_w = network.backprop(x, y)
print(nabla_b)

[array([[ 0.00214254],
       [-0.05287709]])]


In [0]:
print(nabla_w[0])

[[ 0.00214254 -0.05287709]
 [ 0.00428509 -0.10575419]
 [ 0.00642763 -0.15863128]]
